In [1]:
# Import packages
import os
import pandas as pd
import numpy as np
import re 
import networkx as nx
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
import sys

In [2]:
# PROVA
geolocator = Nominatim(user_agent='myapplication')
location = geolocator.geocode("polcenigo")
print(location.raw['lat'],location.raw['lon'])

46.0306995 12.5016263


## Create province dataset

In [14]:
data_file = pd.read_excel('population_province.xlsx',engine='openpyxl')

# Associate to each city a latitude and a longitude
data_file['Province'] = data_file['Province'].str.lower()

province = list(data_file['Province'])
total_pop = list(data_file['Total'])

In [15]:
lat_list = []
lon_list = []

geolocator = Nominatim(user_agent='myapplication')

patience = 1
i = 0 

for prov in province:
    
    if ( i % patience == 0 ):
        sys.stdout.write("\rCompleteness : "+str(round(i/len(province)*100,1))+"%")

    location = geolocator.geocode(prov)
    print(prov)
    lat = location.raw['lat']
    lon = location.raw['lon']
    
    lat_list.append(lat)
    lon_list.append(lon)
    i=i+1

Completeness : 0.0%torino
Completeness : 0.9%vercelli
Completeness : 1.9%novara
Completeness : 2.8%cuneo
Completeness : 3.7%asti
Completeness : 4.7%alessandria
Completeness : 5.6%valle d'aosta
Completeness : 6.5%imperia
Completeness : 7.5%savona
Completeness : 8.4%genova
Completeness : 9.3%la spezia
Completeness : 10.3%varese
Completeness : 11.2%como
Completeness : 12.1%sondrio
Completeness : 13.1%milano
Completeness : 14.0%bergamo
Completeness : 15.0%brescia
Completeness : 15.9%pavia
Completeness : 16.8%cremona
Completeness : 17.8%mantova
Completeness : 18.7%bolzano/bozen
Completeness : 19.6%trento
Completeness : 20.6%verona
Completeness : 21.5%vicenza
Completeness : 22.4%belluno
Completeness : 23.4%treviso
Completeness : 24.3%venezia
Completeness : 25.2%padova
Completeness : 26.2%rovigo
Completeness : 27.1%udine
Completeness : 28.0%gorizia
Completeness : 29.0%trieste
Completeness : 29.9%piacenza
Completeness : 30.8%parma
Completeness : 31.8%reggio nell'emilia
Completeness : 32.7%mode

In [16]:
# Create a dataframe and convert to excel
df = pd.DataFrame(list(zip(province, total_pop, lat_list, lon_list)),columns=['Province','Total','lat','lon'])

df.to_excel('pop_province.xlsx') 



## Create city dataset

In [48]:
# Associate to each city a latitude and a longitude
data_file = pd.read_excel('Elenco-comuni-italiani.xls')
data_file['Denominazione in italiano'] = data_file['Denominazione in italiano'].str.lower()

cities = list(data_file['Denominazione in italiano'])
region_code = list(data_file['Codice Regione'])
region = list(data_file['Denominazione Regione'])
province = list(data_file["Provincia"])
car = list(data_file["Sigla automobilistica"])

In [47]:
lat_list = []
lon_list = []

geolocator = Nominatim(user_agent='myapplication')

patience = 1
i = 0 

for prov in province:
    
    if ( i % patience == 0 ):
        sys.stdout.write("\rCompleteness : "+str(round(i/len(province)*100,1))+"%")

    location = geolocator.geocode(prov)
    lat = location.raw['lat']
    lon = location.raw['lon']
    
    lat_list.append(lat)
    lon_list.append(lon)
    i=i+1

Completeness : 0.0%

In [51]:
# Create a dataframe and convert to excel
df = pd.DataFrame(list(zip(region_code, region, cities, province, car, lat_list, lon_list)),columns=['region_code','region','city','province','car','lat','lon'])

df.to_excel('italian_city.xlsx') 


In [52]:
# Read excel file
df = pd.read_excel('italian_city.xlsx',engine='openpyxl',index_col=0,header=[0])

df

,region_code,region,city,province,car,lat,lon
0,1,Piemonte,agliè,Torino,TO,45.067755,7.682489
1,1,Piemonte,airasca,Torino,TO,45.067755,7.682489
2,1,Piemonte,ala di stura,Torino,TO,45.067755,7.682489
3,1,Piemonte,albiano d'ivrea,Torino,TO,45.067755,7.682489


## Create region dataset

In [ ]:
# Create cities list of Italy
dataset = pd.read_excel('italian_city_dataset.xlsx',engine='openpyxl')
dataset['city'] = dataset['city'].str.lower()
dataset['region'] = dataset['region'].str.lower()

cities = list(dataset['city'])

code_region_list = list(zip(dataset['region_code'], dataset['region']))

# Take unique tuple
code_region_unique = list(set(code_region_list)) 

regions = []
code_regions = []

latitude = list(dataset['lat'])
longitude = list(dataset['lon'])

for i in range(len(code_region_unique)):
    code_regions.append(code_region_unique[i][0])
    regions.append(code_region_unique[i][1])
    
capitals = ['roma','catanzaro','venezia','milano','trento','ancona','cagliari',
            'perugia',"l'aquila",'firenze','bari','aosta','trieste','torino',
            'genova','napoli','potenza','bologna','palermo','campobasso']

df = pd.DataFrame(data=list(zip(code_regions,regions,capitals)),columns=['region_code','region','capitals'])

for cap in capitals:
    mask = df['capitals']==str(cap)
    df.loc[mask, 'lat'] = dataset[dataset['city']==cap]['lat'].values[0]
    df.loc[mask, 'lon'] = dataset[dataset['city']==cap]['lon'].values[0]

df.to_excel('italian_regions_dataset.xlsx') 